<a href="https://colab.research.google.com/github/sam-menezes/combine_system/blob/main/ModelCombine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library

In [ ]:
import pandas as pd
import itertools
import os
import numpy as np
from datetime import datetime
from collections import Counter

# ETL

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/compras.csv", parse_dates=['Data de Embarque'])
df.head()

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
2,1001,2024-06-15,23.0,Xangai
3,1002,2024-06-05,30.0,Shenzhen
4,1002,2024-06-20,35.0,Shenzhen


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Número do Documento  41 non-null     int64         
 1   Data de Embarque     41 non-null     datetime64[ns]
 2   Cubagem (CBM)        41 non-null     float64       
 3   Portos de Embarque   41 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.4+ KB


## Port Analysis

### Plot

In [ ]:
df_port_cbm = df.groupby('Portos de Embarque')['Cubagem (CBM)'].sum()
df_port = df.groupby('Portos de Embarque').size()
print(df_port, df_port_cbm)

Portos de Embarque
Guangzhou           7
Ningbo-Zhoushan    10
Shenzhen           13
Xangai             11
dtype: int64 Portos de Embarque
Guangzhou          338.5
Ningbo-Zhoushan    336.0
Shenzhen           328.0
Xangai             325.5
Name: Cubagem (CBM), dtype: float64


In [ ]:
Porto = df.loc[df['Portos de Embarque'] == 'Xangai']
Porto

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
2,1001,2024-06-15,23.0,Xangai
7,1005,2024-06-30,15.0,Xangai
8,1005,2024-07-05,20.0,Xangai
9,1005,2024-07-10,25.0,Xangai
16,1009,2024-06-10,20.0,Xangai
17,1009,2024-06-15,25.0,Xangai
18,1009,2024-06-20,20.0,Xangai
24,1013,2024-06-01,67.5,Xangai


In [ ]:
# Quais valores já estão no Limite?

Porto_Limite = Porto.loc[Porto['Cubagem (CBM)'] >= 65 ]
Porto_Limite

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
24,1013,2024-06-01,67.5,Xangai
33,1017,2024-06-01,68.0,Xangai


# COMBINE Suggestion



## COMBINE Algorithm


### Port Selection

In [ ]:
portos = df['Portos de Embarque'].unique()
portos

array(['Xangai', 'Shenzhen', 'Ningbo-Zhoushan', 'Guangzhou'], dtype=object)

In [ ]:
df_port = df.loc[df['Portos de Embarque'] == 'Xangai']
df_port.reset_index(inplace=True)
df_port.drop('index', axis=1, inplace=True)

<ipython-input-21-4ecd733d71b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_port.drop('index', axis=1, inplace=True)


In [ ]:
serie_cbm = df_port['Cubagem (CBM)']

### COMBINE 1.0

In [ ]:
def gerar_combinacoes(valores, limite_a_esquerda, limite_a_direita):
    combinacoes = []

    # Iterar sobre cada valor da lista
    for i in range(len(valores)):
        for j in range(i + 1, len(valores)):
            soma = valores[i] + valores[j]
            if soma >= limite_a_esquerda and soma <= limite_a_direita:
                  combinacoes.append((valores[i], valores[j], soma))

    return combinacoes

# Exemplo de uso
valores = serie_cbm
limite_e = 65
limite_d = 67.7
combinacoes = gerar_combinacoes(valores, limite_e, limite_d)

print("Combinacoes válidas (65 > A + B < 67,7):")
for combinacao in combinacoes:
    print(combinacao)

Combinacoes válidas (65 > A + B < 67,7):


### COMBINE 2.0

In [ ]:
# Index de Combine
def gerar_combinacoes_df(df, coluna_cbm, limite_e, limite_d):
    combinacoes_validas = []

    # Verificações preliminares
    assert coluna_cbm in df.columns, f"A coluna '{coluna_cbm}' não existe no DataFrame"
    assert not df.empty, "O DataFrame está vazio"
    assert df[coluna_cbm].notna().all(), "Existem valores nulos na coluna CBM"
    assert pd.api.types.is_numeric_dtype(df[coluna_cbm]), "A coluna CBM deve conter apenas valores numéricos"

    # Gerar combinações de todos os tamanhos possíveis
    for tamanho in range(2, len(df) + 1):
        for indices_combinacao in itertools.combinations(df.index, tamanho):
            try:
                indices_lista = list(indices_combinacao)  # Convertendo a tupla para uma lista
                soma_cbm = df.loc[indices_lista, coluna_cbm].sum()
                print(f"""\n Processando combinação: {indices_combinacao} | Soma: {soma_cbm} \n""")
                if soma_cbm >= limite_e and soma_cbm <= limite_d:
                    combinacoes_validas.append(indices_combinacao)
                    print(f""" \n Combinação válida encontrada: {indices_combinacao} com soma {soma_cbm} \n """)
            except Exception as e:
                print(f"""
                Erro ao processar combinação {indices_combinacao}: {e}
                      """)

    return combinacoes_validas

In [ ]:
# Linhas de Combine Válido
def selecionar_linhas(df, combinacoes):
    combinacoes_linhas = []
    for combinacao in combinacoes:
        try:
            combinacao_df = df.loc[list(combinacao)]  # Convertendo a tupla para uma lista
            combinacoes_linhas.append(combinacao_df)
        except KeyError as e:
            print(f"Erro ao selecionar a combinação {combinacao}: {e}")
    return combinacoes_linhas

In [ ]:
# Teste
display(df_port)

limite_e = 65
limite_d = 67.7
coluna_cbm = 'Cubagem (CBM)'

# Gerar combinações válidas de índices
combinacoes_validas = gerar_combinacoes_df(df_port, coluna_cbm, limite_e, limite_d)

# Selecionar as linhas do DataFrame para cada combinação válida
combinacoes_linhas = selecionar_linhas(df_port, combinacoes_validas)

# Exibir as combinações de linhas
for combinacao in combinacoes_linhas:
    print(combinacao)
    print('-' * 30)

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
2,1001,2024-06-15,23.0,Xangai
3,1005,2024-06-30,15.0,Xangai
4,1005,2024-07-05,20.0,Xangai
5,1005,2024-07-10,25.0,Xangai
6,1009,2024-06-10,20.0,Xangai
7,1009,2024-06-15,25.0,Xangai
8,1009,2024-06-20,20.0,Xangai
9,1013,2024-06-01,67.5,Xangai


A saída de streaming foi truncada nas últimas 5000 linhas.

 Processando combinação: (1, 5, 8, 9) | Soma: 134.5 


 Processando combinação: (1, 5, 8, 10) | Soma: 135.0 


 Processando combinação: (1, 5, 9, 10) | Soma: 182.5 


 Processando combinação: (1, 6, 7, 8) | Soma: 87.0 


 Processando combinação: (1, 6, 7, 9) | Soma: 134.5 


 Processando combinação: (1, 6, 7, 10) | Soma: 135.0 


 Processando combinação: (1, 6, 8, 9) | Soma: 129.5 


 Processando combinação: (1, 6, 8, 10) | Soma: 130.0 


 Processando combinação: (1, 6, 9, 10) | Soma: 177.5 


 Processando combinação: (1, 7, 8, 9) | Soma: 134.5 


 Processando combinação: (1, 7, 8, 10) | Soma: 135.0 


 Processando combinação: (1, 7, 9, 10) | Soma: 182.5 


 Processando combinação: (1, 8, 9, 10) | Soma: 177.5 


 Processando combinação: (2, 3, 4, 5) | Soma: 83.0 


 Processando combinação: (2, 3, 4, 6) | Soma: 78.0 


 Processando combinação: (2, 3, 4, 7) | Soma: 83.0 


 Processando combinação: (2, 3, 4, 8) | Soma: 78.0 


 P

In [ ]:
combinacoes_linhas[0]

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
2,1001,2024-06-15,23.0,Xangai


In [ ]:
len(combinacoes_linhas)

25

In [ ]:
# Todas as caragas consolidadas possíveis

def generate_combinations(df):

    # Ordena o DataFrame inicial pelas colunas desejadas
    df = df.sort_values(by=['Combine_Data_Interval','CBM_Combine'], ascending=[True, False])

    # Lista para armazenar as combinações finais
    combinacoes_finais = []

    # Conjunto para controlar quais índices de linha já foram usados
    used_indices = set()

    # Itera sobre cada linha do DataFrame ordenado
    for index, row in df.iterrows():
        # Verifica se algum índice de linha da combinação atual já foi usado
        if not any(idx in used_indices for idx in row['Cod_Combine']):
            # Adiciona os índices de linha da combinação atual ao conjunto de usados
            used_indices.update(row['Cod_Combine'])
            # Adiciona a combinação atual à lista de combinações finais
            combinacoes_finais.append(row)

    # Retorna o DataFrame final com todas as combinações possíveis
    return pd.DataFrame(combinacoes_finais).reset_index(drop=True)

In [ ]:
def table_goods(df_combine):
  index_del = []
  for c in range(0,len(df_combine)):
    tupla = df_combine['Cod_Combine'][c]
    for i in tupla:
      index_del.append(i)

  return index_del

## Apply COMBINE (test)

In [ ]:
df_port

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
2,1001,2024-06-15,23.0,Xangai
3,1005,2024-06-30,15.0,Xangai
4,1005,2024-07-05,20.0,Xangai
5,1005,2024-07-10,25.0,Xangai
6,1009,2024-06-10,20.0,Xangai
7,1009,2024-06-15,25.0,Xangai
8,1009,2024-06-20,20.0,Xangai
9,1013,2024-06-01,67.5,Xangai


In [ ]:
display(df_port)

limite_e = 65
limite_d = 67.7
coluna_cbm = 'Cubagem (CBM)'

# Gerar combinações válidas de índices
combinacoes_validas = gerar_combinacoes_df(df_port, coluna_cbm, limite_e, limite_d)

# Selecionar as linhas do DataFrame para cada combinação válida
stack_df_combine = selecionar_linhas(df_port, combinacoes_validas)

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
2,1001,2024-06-15,23.0,Xangai
3,1005,2024-06-30,15.0,Xangai
4,1005,2024-07-05,20.0,Xangai
5,1005,2024-07-10,25.0,Xangai
6,1009,2024-06-10,20.0,Xangai
7,1009,2024-06-15,25.0,Xangai
8,1009,2024-06-20,20.0,Xangai
9,1013,2024-06-01,67.5,Xangai


A saída de streaming foi truncada nas últimas 5000 linhas.
 Processando combinação: (1, 3, 4, 6) | Soma: 77.0 


 Processando combinação: (1, 3, 4, 7) | Soma: 82.0 


 Processando combinação: (1, 3, 4, 8) | Soma: 77.0 


 Processando combinação: (1, 3, 4, 9) | Soma: 124.5 


 Processando combinação: (1, 3, 4, 10) | Soma: 125.0 


 Processando combinação: (1, 3, 5, 6) | Soma: 82.0 


 Processando combinação: (1, 3, 5, 7) | Soma: 87.0 


 Processando combinação: (1, 3, 5, 8) | Soma: 82.0 


 Processando combinação: (1, 3, 5, 9) | Soma: 129.5 


 Processando combinação: (1, 3, 5, 10) | Soma: 130.0 


 Processando combinação: (1, 3, 6, 7) | Soma: 82.0 


 Processando combinação: (1, 3, 6, 8) | Soma: 77.0 


 Processando combinação: (1, 3, 6, 9) | Soma: 124.5 


 Processando combinação: (1, 3, 6, 10) | Soma: 125.0 


 Processando combinação: (1, 3, 7, 8) | Soma: 82.0 


 Processando combinação: (1, 3, 7, 9) | Soma: 129.5 


 Processando combinação: (1, 3, 7, 10) | Soma: 130.0 


 Processand

In [ ]:
combinacoes_validas

[(0, 1, 2),
 (0, 1, 5),
 (0, 1, 7),
 (0, 4, 5),
 (0, 4, 7),
 (0, 5, 6),
 (0, 5, 8),
 (0, 6, 7),
 (0, 7, 8),
 (1, 2, 4),
 (1, 2, 6),
 (1, 2, 8),
 (1, 4, 5),
 (1, 4, 7),
 (1, 5, 6),
 (1, 5, 8),
 (1, 6, 7),
 (1, 7, 8),
 (3, 5, 7),
 (4, 5, 6),
 (4, 5, 8),
 (4, 6, 7),
 (4, 7, 8),
 (5, 6, 8),
 (6, 7, 8)]

In [ ]:
combinations = len(stack_df_combine)
f"Foram feitas {combinations} combinações válidas"

'Foram feitas 25 combinações válidas'

In [ ]:
stack_df_combine[2]

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
0,1001,2024-06-01,20.0,Xangai
1,1001,2024-06-10,22.0,Xangai
7,1009,2024-06-15,25.0,Xangai


### Best Combinations

In [ ]:
Combine_Index = []
CBM_Combine = []
Combine_DOC = []
Combine_Data_Interval = []

In [ ]:
for i in range (0, combinations):
  Combine_Index.append(i)
  CBM_Combine.append(stack_df_combine[i]['Cubagem (CBM)'].sum())
  interval = stack_df_combine[i]['Data de Embarque'].max() - stack_df_combine[i]['Data de Embarque'].min()
  Combine_Data_Interval.append(interval.days)

In [ ]:
df_base = pd.DataFrame()
df_base['Combine_Index'] = Combine_Index
df_base['CBM_Combine'] = CBM_Combine
df_base['Combine_Data_Interval'] = Combine_Data_Interval
df_base['Cod_Combine'] = combinacoes_validas

In [ ]:
df_base = df_base.query('Combine_Data_Interval <= 30').reset_index(drop=True)

In [ ]:
df_base = df_base.sort_values(by=['Combine_Data_Interval','CBM_Combine'], ascending=[True, False]).reset_index(drop=True)

In [ ]:
df_final = generate_combinations(df_base)
df_final

,Combine_Index,CBM_Combine,Combine_Data_Interval,Cod_Combine
0,16,67.0,5,"(1, 6, 7)"
1,20,65.0,20,"(4, 5, 8)"


In [ ]:
stack_df_combine[16]

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
1,1001,2024-06-10,22.0,Xangai
6,1009,2024-06-10,20.0,Xangai
7,1009,2024-06-15,25.0,Xangai


In [ ]:
stack_df_combine[20]

,Número do Documento,Data de Embarque,Cubagem (CBM),Portos de Embarque
4,1005,2024-07-05,20.0,Xangai
5,1005,2024-07-10,25.0,Xangai
8,1009,2024-06-20,20.0,Xangai


In [ ]:
df_final

,Combine_Index,CBM_Combine,Combine_Data_Interval,Cod_Combine
0,16,67.0,5,"(1, 6, 7)"
1,20,65.0,20,"(4, 5, 8)"


# Apply COMBINE

## Porto


In [ ]:
portos = df['Portos de Embarque'].unique()
df_port = df.loc[df['Portos de Embarque'] == portos[0]]
df_port.reset_index(inplace=True)
df_port.drop('index', axis=1, inplace=True)
portos

<ipython-input-137-7c9d51baceb5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_port.drop('index', axis=1, inplace=True)


array(['Xangai', 'Shenzhen', 'Ningbo-Zhoushan', 'Guangzhou'], dtype=object)

## Apply Combination

## Best Dataframe

In [ ]:
def save_combinations(df, e=65, d=67.7):

  portos = df['Portos de Embarque'].unique()
  #

  for porto in portos:
    df_port = df.loc[df['Portos de Embarque'] == porto]
    df_port.reset_index(inplace=True)
    df_port.drop('index', axis=1, inplace=True)

    #
    coluna_cbm = 'Cubagem (CBM)'

    # Gerar combinações válidas de índices
    combinacoes_validas = gerar_combinacoes_df(df_port, coluna_cbm, e, d)

    # Selecionar as linhas do DataFrame para cada combinação válida
    stack_df_conbine = selecionar_linhas(df_port, combinacoes_validas)

    #

    Conbine_Index = []
    CBM_Combine = []
    Combine_DOC = []
    Combine_Data_Interval = []

    #

    combinations = len(stack_df_conbine)

    #

    for i in range (0, combinations):
      Conbine_Index.append(i)
      CBM_Combine.append(stack_df_conbine[i]['Cubagem (CBM)'].sum())
      Combine_DOC.append(porcentagem_elementos_iguais(stack_df_conbine[i]['Número do Documento'].tolist()))
      interval = stack_df_conbine[i]['Data de Embarque'].max() - stack_df_conbine[i]['Data de Embarque'].min()
      Combine_Data_Interval.append(interval.days)

    #

    df_base = pd.DataFrame()
    df_base['Combine_Index'] = Conbine_Index
    df_base['CBM_Combine'] = CBM_Combine
    df_base['Combine_DOC'] = Combine_DOC
    df_base['Combine_data_interval'] = Combine_Data_Interval
    df_base['Cod_Combine'] = combinacoes_validas

    #

    df_base = df_base.query('Combine_Data_Interval <= 30').reset_index(drop=True)
    df_base = df_base.sort_values(by=['Combine_Data_Interval','CBM_Combine'], ascending=[True, False]).reset_index(drop=True)

    #

    df_best = generate_combinations(df_base)
    df_best.to_csv(f'/content/drive/MyDrive/Colab Notebooks/combination/{porto}/best/best_table.csv', index=False)
    containers = len(df_best)

    #

    for c in range (0, containers):
      ConbineIndex = df_best['Conbine_Index'][c]
      stack_df_conbine[ConbineIndex].to_csv(f'/content/drive/MyDrive/Colab Notebooks/combination/containers/goods_{c}.csv', index=False)

    tabelas = os.listdir('/content/drive/MyDrive/Colab Notebooks/combination/containers')

    print(f"Para o porto {porto}, há {len(tabelas)} tabelas de containers consolidados salvas")
